In [1]:
import numpy as np
from scipy.stats import poisson
from functools import lru_cache
from tqdm import tqdm

In [3]:
ACTIONS = np.arange(-5, 6)
MAX_CARS_LOC = 20
N_CARS_PER_LOC = (MAX_CARS_LOC + 1)    #[0, 20]
N_STATES = N_CARS_PER_LOC * N_CARS_PER_LOC
CAR_RENT_COST = 10
CAR_MOVE_COST = 2

LAMBDA_RET_LOC1 = 3
LAMBDA_RET_LOC2 = 2
LAMBDA_RNT_LOC1 = 3
LAMBDA_RNT_LOC2 = 4

MAX_POISSON_OUTCOME = 11
GAMMA = 0.9
THETA = 1e-4

#Pre-computing poisson pmf due to high computational cost
poisson_pmf_req1 = [poisson.pmf(i, LAMBDA_RNT_LOC1) for i in range(MAX_POISSON_OUTCOME)]
poisson_pmf_req2 = [poisson.pmf(i, LAMBDA_RNT_LOC2) for i in range(MAX_POISSON_OUTCOME)]
poisson_pmf_ret1 = [poisson.pmf(i, LAMBDA_RET_LOC1) for i in range(MAX_POISSON_OUTCOME)]
poisson_pmf_ret2 = [poisson.pmf(i, LAMBDA_RET_LOC2) for i in range(MAX_POISSON_OUTCOME)]

@lru_cache(maxsize=None)
def calculate_expected_val(state, action): #R(s, a) | P(s' |s, a)
    #Estado do dia anterior
    n1_end_prev, n2_end_prev = divmod(state, N_CARS_PER_LOC)

    #Inicializa recompensa e executa a ação, que é mover os carros de uma localização para outra
    #Valores positivos loc1 -> loc2
    #Valores negativos loc2 -> loc1
    expected_total_reward = 0.0

    n1_after_move = n1_end_prev - action
    n2_after_move = n2_end_prev + action

    if not (0 <= n1_after_move <= MAX_CARS_LOC) or not (0 <= n2_after_move <= MAX_CARS_LOC):
        return 0.0, np.zeros(N_STATES)

    #Inicializa a probabilidade dos estados seguintes
    next_state_prob = np.zeros(N_STATES)

    #Verifica os limites para n1 e n2 se manterem entre 0 e 20
    n1_start_day = min(max(0, n1_after_move), MAX_CARS_LOC)
    n2_start_day = min(max(0, n2_after_move), MAX_CARS_LOC)

    #Custo por mover cada carro
    move_cost = abs(action) * CAR_MOVE_COST

    #Como o processo é estocástico, é necessário verificar todas as possibilidades para encontrar a probabilidade de cair no próximo estado
    for req1 in range(MAX_POISSON_OUTCOME):
        prob_req1 = poisson_pmf_req1[req1] 
        for req2 in range(MAX_POISSON_OUTCOME):
            prob_req2 = poisson_pmf_req2[req2]
            for ret1 in range(MAX_POISSON_OUTCOME):
                prob_ret1 = poisson_pmf_ret1[ret1]
                for ret2 in range(MAX_POISSON_OUTCOME):
                    prob_ret2 = poisson_pmf_ret2[ret2]
                    

                    prob_joint_outcome = prob_req1 * prob_req2 * prob_ret1 * prob_ret2 #Probabilidade conjunta (São independentes)

                    # Calcula o número de carros alugados. Se houverem mais clientes do que carros, aluga todos os carros restantes.
                    rented_1 = min(n1_start_day, req1)
                    rented_2 = min(n2_start_day, req2)
                    
                    # Calcula a recompensa obtida do aluguel das 2 localizações menos o custo de mover os carros
                    rental_income = (rented_1 + rented_2) * CAR_RENT_COST
                    current_reward = rental_income - move_cost
                    
                    # Esperança da recompensa
                    expected_total_reward += prob_joint_outcome * current_reward

                    # Calcula os carros do próximo dia
                    cars_after_rentals_1 = n1_start_day - rented_1
                    cars_after_rentals_2 = n2_start_day - rented_2

                    # Retornam os carros do "dia anterior"
                    next_n1 = min(MAX_CARS_LOC, cars_after_rentals_1 + ret1)
                    next_n2 = min(MAX_CARS_LOC, cars_after_rentals_2 + ret2)
                    
                    #Atribui probabilidade para o próximo estado obtido
                    next_s_idx = next_n1 * N_CARS_PER_LOC + next_n2
                    next_state_prob[next_s_idx] += prob_joint_outcome

    #Normaliza as probabilidades
    sum_probs = np.sum(next_state_prob)
    if sum_probs > 0:
        next_state_prob /= sum_probs

    #Retorna a recompensa esperada de cada próximo estado R(s,a) e sua probabilidade P(s' |s, a)
    return expected_total_reward, next_state_prob

def get_prob_policy(policy, state, action):
    if action in policy[state]:
        return 1/len(policy[state])

    return 0

def policy_eval(state_values, policy, theta, gamma):
    while True:
        delta = 0
        v_old = state_values.copy()
        for state in tqdm(range(N_STATES), desc="Policy Evaluation"):
            v_new = 0
            for action in ACTIONS:
                expected_r, next_state_prob = calculate_expected_val(state, action) # R(s,a) , p(s' |s, a)
                v_action = expected_r + gamma * np.dot(next_state_prob, state_values)
                v_new += v_action * get_prob_policy(policy, state, action)

            state_values[state] = v_new

            delta = max(delta, abs(v_old[state] - v_new))

        if delta < theta:
            break


def policy_improv(policy, state_values, gamma):
    pol_stable = True
    old_policy = [i.copy() for i in policy]
    for state in tqdm(range(N_STATES), desc="Policy Improvement"):
        best_q = float("-inf")
        best_action = 0

        for action in ACTIONS:
            expected_r, next_state_prob = calculate_expected_val(state, action) # R(s,a) , p(s' |s, a)

            if np.sum(next_state_prob) == 0:
                continue

            q_sa = 0
            for next_s in range(N_STATES):
                q_sa += next_state_prob[next_s] * (expected_r + gamma * state_values[next_s])

            if best_q < q_sa:
                best_q = q_sa
                best_action = action

        policy[state] = [best_action]
        if policy[state] != old_policy[state]:
            pol_stable = False

    return pol_stable

def policy_iter(policy, state_values, gamma):
    iterations = 0
    while True:
        iterations += 1
        print("Política atual: ")
        print(np.array(policy).reshape(21,21))
        policy_eval(state_values, policy, 1e-3, gamma)
        pol_stable = policy_improv(policy, state_values, gamma)
        if pol_stable:
            break
    print("Iterations: ", iterations)

policy = [[0] for s in range(N_STATES)]
state_values = np.zeros(N_STATES)

policy_iter(policy, state_values, 0.9)
print(np.array(policy).reshape(21,21))

Política atual: 
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


Policy Evaluation:   0%|          | 0/441 [00:00<?, ?it/s]

Policy Improvement: 100%|██████████| 441/441 [00:01<00:00, 331.01it/s]


Política atual: 
[[ 0  0  0  0  0  0  0  0 -1 -1 -1 -2 -2 -2 -2 -3 -3 -3 -3 -3 -4]
 [ 0  0  0  0  0  0  0  0  0  0 -1 -1 -1 -1 -2 -2 -2 -2 -2 -3 -3]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 -1 -1 -1 -1 -1 -2 -2 -2]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1 -1 -1 -1]
 [ 1  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1]
 [ 2  2  2  1  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 3  3  2  2  2  2  1  1  1  1  0  0  0  0  0  0  0  0  0  0  0]
 [ 4  3  3  3  3  2  2  2  2  1  1  1  0  0  0  0  0  0  0  0  0]
 [ 4  4  4  4  3  3  3  3  2  2  2  1  1  1  1  0  0  0  0  0  0]
 [ 5  5  5  4  4  4  4  3  3  3  2  2  2  2  1  1  1  0  0  0  0]
 [ 5  5  5  5  5  5  4  4  4  3  3  3  3  2  2  2  1  1  0  0  0]
 [ 5  5  5  5  5  5  5  5  4  4  4  4  3  3  3  2  2  1  1  0  0]
 [ 5  5  5  5  5  5  5  5  5  5  5  4  4  4  3  3  2  2  1  0  0]
 [ 5  5  5  5  5  5  5  5  5  5  5  5  5  4  4  3  3  2  1  1  0]
 [ 5  5  5  5  5  5  5  5  5  5  5  5  5  5  4  4  3  2  2 

Policy Improvement: 100%|██████████| 441/441 [00:01<00:00, 335.15it/s]


...

Política atual: 
[[ 0  0  0  0  0  0  0  0 -1 -1 -2 -2 -2 -3 -3 -3 -3 -3 -4 -4 -4]
 [ 0  0  0  0  0  0  0  0  0 -1 -1 -1 -2 -2 -2 -2 -2 -3 -3 -3 -3]
 [ 0  0  0  0  0  0  0  0  0  0  0 -1 -1 -1 -1 -1 -2 -2 -2 -2 -2]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1 -1 -1 -1 -1 -2]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1 -1]
 [ 1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 2  2  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 3  2  2  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 3  3  2  2  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 4  3  3  2  2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 4  4  3  3  2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 5  4  4  3  2  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 5  5  4  3  2  2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 5  5  4  3  3  2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 5  5  4  4  3  2  1  0  0  0  0  0  0  0  0  0  0  0  0 

Policy Improvement: 100%|██████████| 441/441 [00:01<00:00, 313.09it/s]


Política atual: 
[[ 0  0  0  0  0  0  0  0 -1 -1 -2 -2 -2 -3 -3 -3 -3 -3 -4 -4 -4]
 [ 0  0  0  0  0  0  0  0  0 -1 -1 -1 -2 -2 -2 -2 -2 -3 -3 -3 -3]
 [ 0  0  0  0  0  0  0  0  0  0  0 -1 -1 -1 -1 -1 -2 -2 -2 -2 -2]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1 -1 -1 -1 -1 -2]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1 -1]
 [ 1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 2  2  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 3  2  2  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 3  3  2  2  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 4  3  3  2  2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 4  4  3  3  2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 5  4  4  3  2  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 5  5  4  3  2  2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 5  5  4  3  3  2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 5  5  4  4  3  2  1  0  0  0  0  0  0  0  0  0  0  0  0 

Policy Improvement: 100%|██████████| 441/441 [00:01<00:00, 333.14it/s]

Iterations:  5
[[ 0  0  0  0  0  0  0  0 -1 -1 -2 -2 -2 -3 -3 -3 -3 -3 -4 -4 -4]
 [ 0  0  0  0  0  0  0  0  0 -1 -1 -1 -2 -2 -2 -2 -2 -3 -3 -3 -3]
 [ 0  0  0  0  0  0  0  0  0  0  0 -1 -1 -1 -1 -1 -2 -2 -2 -2 -2]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1 -1 -1 -1 -1 -2]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1 -1]
 [ 1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 2  2  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 3  2  2  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 3  3  2  2  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 4  3  3  2  2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 4  4  3  3  2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 5  4  4  3  2  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 5  5  4  3  2  2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 5  5  4  3  3  2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 5  5  4  4  3  2  1  0  0  0  0  0  0  0  0  0  0  0  0  0